In [ ]:
import math
import numpy as np
from numpy.random import seed
import random
import pandas as pd

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

import tensorflow as tf
import tensorflow.keras.utils as Utils
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras import utils as np_utils

In [ ]:
# Seed Everything to reproduce results for future use cases
def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)
    
    # Seed value for NumPy
    np.random.seed(seed)
    
    # Seed value for Python's random library
    random.seed(seed)
    
    # Force TensorFlow to use single thread
    # Multiple threads are a potential source of non-reproducible results.
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    # Make sure that TensorFlow uses a deterministic operation wherever possible
    tf.compat.v1.set_random_seed(seed)

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

seed_everything()

In [ ]:
#import functions
from global_functions import *

## Config and load images for models training

In [ ]:
#Train only the emotions present in all datasets (training and testing)
emotions_labels = {0:'raiva',  1:'medo', 2:'alegria', 3: 'tristeza'}
class_names = ['raiva', 'medo', 'alegria', 'tristeza']

In [ ]:
img_width = 224
img_height = 224
img_depth = 3
num_classes = len(class_names)
print('Classes:' + str(num_classes))

In [ ]:
#emotions_labels = {0:'raiva', 1:'aversão', 2:'medo', 3:'alegria', 4: 'tristeza', 5: 'surpresa', 6: 'desprezo'}
#directories <dataset>_<emotion>, for ex., NIMH_Angry (emotion:[Angry, Happy, Sad, Fear])

#Replace value of DATASET variable for each available dataset
DATASET = 'NIMH'
filesStru = getFiles(dataset=DATASET, crop=True, extensao='jpg')

files = filesStru.files
files_class = filesStru.files_class

In [ ]:
#images for emotion
data = {'imagem': files, 'emocao': files_class}
df = pd.DataFrame(data)
df['emocao'].value_counts().sort_index(ascending=True)

In [ ]:
#% images for emotion
df['emocao'].value_counts(normalize=True, ascending=False) #.sort_index(ascending=True)

In [ ]:
df['emocao'].hist()

In [ ]:
#Show some_images
some_images(files, files_class, emotions_labels)

In [ ]:
#all_images(files, files_class, emotions_labels)

In [ ]:
img_features_source = []
img_features_source = create_features(files, img_width, img_height)

In [ ]:
#Show image from img_features
pyplot.imshow(img_features_source[93].astype(np.uint8));

In [ ]:
img_labels = np_utils.to_categorical(files_class,num_classes)
img_labels.shape

## Split data into Training (training and validation) and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_features_source, img_labels, shuffle = True, 
                                                      stratify = img_labels, test_size = 0.20, random_state = 42)

X_train, X_valid, y_train, y_valid  = train_test_split(X_train, y_train, shuffle = True, 
                       test_size = 0.25, random_state = 42)
                       
X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape
        
np.save('X_train_' + DATASET + '.npy', X_train) #salvar dados treino
np.save('Y_train_' + DATASET + '.npy', y_train) 

np.save('X_valid_' + DATASET + '.npy', X_train) #salvar dados de validação
np.save('Y_valid_' + DATASET + '.npy', y_train) 

np.save('X_test_' + DATASET + '.npy', X_test) #salvar dados teste 
np.save('Y_test_' + DATASET + '.npy', y_test) 

print(X_train.shape, X_test.shape, X_valid.shape)